# Single Amino Acid Language Model

In [35]:
import torch
import torch.nn as nn
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
from IPython.display import HTML, display

In [2]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 

In [3]:
dev

'cpu'

In [4]:
torch.manual_seed(42)

In [36]:
HTML("""
<style>
table, th, td {
  border: 1px solid black;
}
</style>
""")

## Loading Language Model data

In [5]:
# If trained in colab
from google.colab import drive

drive.mount('content/', force_remount=True)

file_paths = Path('/content/content/MyDrive/subcellular-location/v2/')

ModuleNotFoundError: No module named 'google.colab'

In [6]:
# If trained from home (linux)
file_paths = Path('/home/mees/Desktop/Machine_Learning/subcellular_location/')

In [7]:
# Load data in a pandas dataframe
data_file = file_paths / Path('data/raw/LM_data_2021-03-11.csv')
df = pd.read_csv(data_file, sep=';')
df.head()

,Entry,Entry name,Sequence
0,P68307,NU3M_BALMU,MNLLLTLLTNTTLALLLVFIAFWLPQLNVYAEKTSPYECGFDPMGS...
1,P0CY61,O162_CONBU,MKLTCVLIIAVLFLTAITADDSRDKQVYRAVGLIDKMRRIRASEGC...
2,Q0VIL3,OTOMP_DANRE,MDLPGGHLAVVLFLFVLVSMSTENNIIRWCTVSDAEDQKCLDLAGN...
3,A1W9I4,NUSB_ACISJ,MTDSTHPTPSARPPRQPRTGTTGTGARKAGSKSGRSRAREFALQAL...
4,Q8DBX0,OMPU_VIBVU,MKKTLIALSVSAAAVATGVNAAELYNQDGTSLDMGGRAEARLSMKD...


In [8]:
# Delete entry and entry name columns since we do not need it for the language model
df.drop(['Entry', 'Entry name'], axis = 1, inplace=True)

## Tokenize the sequences

Use a function to tokenize the sequences. In this case, we use a kmer size of 1, which are single amino acids. And we add the End of Sequence (EOS) tag.

In [9]:
def create_vocab(df, protein_seqs_column, kmer_sz, stride=1, eos_token=True):
    kmers = set()
        
    # Map kmers for one-hot encoding
    kmer_to_id = dict()
    id_to_kmer = dict()

    # Loop over the protein sequences
    for protein_seq in df[protein_seqs_column]:
        # Loop over the sequence and add the amino acid if it is not in kmers set.
        seq_len = len(protein_seq)


        for i in range(0, seq_len - (kmer_sz - 1), stride):

            kmer = protein_seq[i: i + kmer_sz]

            if kmer not in list(kmers):
                ind = len(kmers)
                kmers.add(kmer)

                # Also create the dictionary
                kmer_to_id[kmer] = ind
                id_to_kmer[ind] = kmer

    if eos_token:
        token = '<EOS>'
        ind = len(kmers)
        
        kmers.add(token)

        # Also create the dictionary
        kmer_to_id[token] = ind
        id_to_kmer[ind] = token

    vocab_sz = len(kmers)

    assert vocab_sz == len(kmer_to_id.keys())
    
    return kmer_to_id, id_to_kmer, vocab_sz

In [10]:
def tokenize(df, protein_seqs_column, kmer_sz, stride=1, eos_token=True, premade_vocab=False):
    
    
    # Create the vocabulary
    if not premade_vocab:
        
        kmer_to_id, id_to_kmer, vocab_sz = create_vocab(df, protein_seqs_column, kmer_sz, stride, eos_token)
                
    else:
        kmer_to_id, id_to_kmer = premade_vocab
        vocab_sz = len(kmer_to_id)
            
    # Tokenize the sequences in the DF

    tokenized = []
    for i, protein_seq in enumerate(df[protein_seqs_column], 0):
        sequence = []
        
        # If the kmer can't be found these indexes should be deleted
        remove_idxs = []
        
        # Loop over the protein sequence
        for i in  range(len(protein_seq) - (kmer_sz -1)):
            # Convert kmer to integer
            kmer = protein_seq[i: i + kmer_sz]
            
            # For some reason, some kmers miss. Thus these sequences have to be removed
            try:
                sequence.append(kmer_to_id[kmer])
            except:
                remove_idxs.append(i)
                
        if eos_token:
            sequence.append(kmer_to_id['<EOS>'])
            
        tokenized.append(sequence)
            
    df['tokenized_seqs'] = tokenized
    
    df.drop(remove_idxs, inplace=True)
    
    return df, vocab_sz, kmer_to_id, id_to_kmer

In [11]:
KMER_SIZE = 1 # Single Amino Acids

In [12]:
# Tokenize the protein sequence
df, vocab_sz, kmer_to_id, id_to_kmer = tokenize(df, 'Sequence', KMER_SIZE)

In [39]:
save_vocab_file = file_paths / Path('data/interim/AA_vocab.pkl')
pickle.dump([kmer_to_id, id_to_kmer], open(save_vocab_file, 'wb'))

In [13]:
df.head()

,Sequence,tokenized_seqs
0,MNLLLTLLTNTTLALLLVFIAFWLPQLNVYAEKTSPYECGFDPMGS...,"[0, 1, 2, 2, 2, 3, 2, 2, 3, 1, 3, 3, 2, 4, 2, ..."
1,MKLTCVLIIAVLFLTAITADDSRDKQVYRAVGLIDKMRRIRASEGC...,"[0, 13, 2, 3, 15, 5, 2, 7, 7, 4, 5, 2, 6, 2, 3..."
2,MDLPGGHLAVVLFLFVLVSMSTENNIIRWCTVSDAEDQKCLDLAGN...,"[0, 17, 2, 9, 16, 16, 19, 2, 4, 5, 5, 2, 6, 2,..."
3,MTDSTHPTPSARPPRQPRTGTTGTGARKAGSKSGRSRAREFALQAL...,"[0, 3, 17, 14, 3, 19, 9, 3, 9, 14, 4, 18, 9, 9..."
4,MKKTLIALSVSAAAVATGVNAAELYNQDGTSLDMGGRAEARLSMKD...,"[0, 13, 13, 3, 2, 7, 4, 2, 14, 5, 14, 4, 4, 4,..."


In [14]:
data = []
sequences = df['tokenized_seqs'].tolist()
for seq in sequences:
    for kmer in seq:
        data.append(int(kmer))
        
print(data[:10])

[0, 1, 2, 2, 2, 3, 2, 2, 3, 1]


## Dataset

In [15]:
class AminoLMDataset(torch.utils.data.Dataset):
    def __init__(self, data, seq_len, stride=1):
        self.data = torch.Tensor(data)
        self.seq_len = seq_len
        self.stride = stride
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        xs = torch.LongTensor(data[idx: idx + self.seq_len])
        targets = data[idx + self.stride: idx + self.seq_len + self.stride]

        ys = []

        for target in targets:
            y = torch.tensor(target)
            ys.append(y)

        ys = torch.stack(ys)

        ys = ys.to(dev)
        xs = xs.to(dev) 
    
        return xs, ys

## Dropouts

In [16]:
class EmbeddingDropout(nn.Module):
    "Apply dropout to an Embedding with probability emp_p"

    def __init__(self, emb_p=0):
        super(EmbeddingDropout, self).__init__()
        
        self.emb_p = emb_p

    def forward(self, inp):
       
        drop = torch.nn.Dropout(self.emb_p)
        placeholder = torch.ones((inp.size(1), 1)).to(dev)
        mask = drop(placeholder)      
        out = inp * mask
        
        return out

In [30]:
class AWD_LSTM(torch.nn.Module):
    def __init__(self, num_layers, vocab_sz, emb_dim, hid_sz, hidden_p, embed_p, input_p, weight_p, batch_sz = 1):
        super(AWD_LSTM, self).__init__()
        
        # Embedding with droput
        self.encoder = nn.Embedding(vocab_sz, emb_dim)
        self.emb_drop = EmbeddingDropout(emb_p=embed_p)

        
        # Dropouts on the inputs and the hidden layers
        self.hid_dp = torch.nn.Dropout(p=hidden_p)
        
        self.lstms = nn.LSTM(emb_dim, hid_sz, num_layers, batch_first = True, dropout=input_p)

        
        # Save all variables        
        self.num_layers = num_layers
        self.vocab_sz = vocab_sz
        self.emb_dim = emb_dim
        self.hid_sz = hid_sz
        self.hidden_p = hidden_p
        self.embed_p = embed_p
        self.input_p = input_p
        self.weight_p = weight_p
        self.batch_sz = batch_sz

        # Initialize hidden layers        
        self.reset_hidden()
        self.last_hiddens = (self.hidden_state, self.cell_state)
                
    def forward(self, xs):
        """Forward pass AWD-LSTM""" 
        
        bs, sl = xs.shape

        ys = []
        
        hiddens = self.last_hiddens
        
        # Embed the inputs
        embed = self.encoder(xs)
        embed_dp = self.emb_drop(embed)
        
        inp = embed_dp.view(bs, sl, -1)
        
        # Dropout on hidden layers
        hiddens_dp = []
        for hidden_state in hiddens:
            hiddens_dp.append(self.hid_dp(hidden_state))
            
        hiddens_dp = tuple(hiddens_dp)
        
        output, hidden_outs = self.lstms(embed_dp.view(sl, bs, -1), hiddens_dp)
        
        # Detach hidden layers
        hidden_detached = []
        
        for hidden_out in hidden_outs:
            hidden_detached.append(hidden_out.detach())
            
        
        self.last_hiddens = hidden_detached
        
        return output
    
    def reset_hidden(self):
        self.hidden_state = torch.zeros((self.num_layers, self.num_layers, self.hid_sz)).to(dev)
        self.cell_state = torch.zeros((self.num_layers, self.num_layers, self.hid_sz)).to(dev)
        self.last_hiddens = (self.hidden_state, self.cell_state)
    
    def freeze_to(self , n):
        
        params_to_freeze = n * 4 + 1 # Since each LSTM layer has 4 parameters plus 1 to also freeze the encoder
        
        total_params = len(list(self.parameters()))
        
        for i, parameter in enumerate(self.parameters()):
            parameter.requires_grad = True
            
            if i < params_to_freeze:
                parameter.requires_grad = False
            
            
        for name, parameter in self.named_parameters():
            print(name)
            print(parameter.requires_grad)

In [18]:
class AALM(nn.Module):
    def __init__(self, num_layers, vocab_sz, emb_dim, hid_sz, hidden_p, embed_p, input_p, weight_p, batch_sz = 1):
        super(AALM, self).__init__()
        
        self.encoder = AWD_LSTM(num_layers, vocab_sz, emb_dim, hid_sz, hidden_p, 
                                embed_p, input_p, weight_p, batch_sz=batch_sz)
        self.decoder = nn.Linear(hid_sz, vocab_sz)
        
    def forward(self, inp):
        
        encoded = self.encoder(inp)
        
        y = self.decoder(encoded)
        
        return y 
    
    def freeze_to(self, n):
        self.encoder.freeze_to(n)
        
    def reset_hidden(self):
        self.encoder.reset_hidden()

## Create model

In [19]:
# Hyperparameters
emb_dim = 10 # Embeddding dimension
hid_sz = 1150 # Hidden size
num_layers = 20 # Number of LSTM layers stacked together
seq_len = num_layers
bs = 8

# Dropout parameters

embed_p = 0.1 # Dropout probability on the embedding
hidden_p = 0.3 # Dropout probability on hidden-to-hidden weight matrices
# Dropout tussen de inputs van de LSTMs moet ik er nog in bouwen
input_p = 0.3 # Dropout probablity on the LSTM input between LSTMS
weight_p = 0.5 # Dropout probability on LSTM-to-LSTM weight matrices

In [31]:
model = AALM(num_layers, vocab_sz, emb_dim, hid_sz, hidden_p, embed_p, input_p, weight_p, batch_sz=bs)
model = model.to(dev)
model

AALM(
  (encoder): AWD_LSTM(
    (encoder): Embedding(26, 10)
    (emb_drop): EmbeddingDropout()
    (hid_dp): Dropout(p=0.3, inplace=False)
    (lstms): LSTM(10, 1150, num_layers=20, batch_first=True, dropout=0.3)
  )
  (decoder): Linear(in_features=1150, out_features=26, bias=True)
)

## Training the model

In [21]:
training_set = AminoLMDataset(data, seq_len)

In [22]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=bs, shuffle=False)

In [23]:
total_train_len = len(training_loader)
total_train_len

7374044

In [24]:
# Hyperparamaters
learning_rate = 0.001
epochs = 10

In [25]:
# Costfunction and optimize algorithm
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [33]:
# Test for the real work
for i, entry in enumerate(training_loader, 0):
    xs, ys = entry[0], entry[1]

    print('Input shape:')
    print(xs.shape)

    outputs = model(xs)
    
    bs, sl = outputs.shape[:2]
    
    # Flatten the output
    outputs = outputs.view(bs * sl, -1)
    
    print('Labels:')  
    print(ys)
    
    # Flatten the label
    ys = ys.view(-1)
    
    print('Output shape:')
    print(outputs.shape)
    
    print('Label shape:')
    print(ys.shape)
    
    loss = criterion(outputs, ys)
    print(loss)
    
    break

Input shape:
torch.Size([8, 20])
Labels:
tensor([[ 1,  2,  2,  2,  3,  2,  2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,
          7,  4],
        [ 2,  2,  2,  3,  2,  2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,
          4,  6],
        [ 2,  2,  3,  2,  2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,  4,
          6,  8],
        [ 2,  3,  2,  2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,  4,  6,
          8,  2],
        [ 3,  2,  2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,  4,  6,  8,
          2,  9],
        [ 2,  2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,  4,  6,  8,  2,
          9, 10],
        [ 2,  3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,  4,  6,  8,  2,  9,
         10,  2],
        [ 3,  1,  3,  3,  2,  4,  2,  2,  2,  5,  6,  7,  4,  6,  8,  2,  9, 10,
          2,  1]])
Output shape:
torch.Size([160, 26])
Label shape:
torch.Size([160])
tensor(3.2592, grad_fn=<NllLossBackward>)


In [38]:
display(HTML(
    """<table>
        <thead>
          <tr>
          <th>Epoch</th>
          <th>Percentage</th>
          <th>Loss</th>
          <th>Accuracy</th>
          <th>Time</th>
          </tr>
        </thead>
        <tbody>
        """
))

for epoch in range(epochs):

    start_time = time.time()

    model.reset_hidden()

    # Initialize loss at 0
    epoch_loss = 0.0
    iteration_loss = 0.0
    train_total = 0

    for i, entry in enumerate(training_loader, 0):

        if len(entry[0]) == bs:

            model.zero_grad()

            xs, ys = entry[0], entry[1]

            outputs = model(xs)

            bs, sl = outputs.shape[:2]

            # Flatten the output
            outputs = outputs.view(bs * sl, -1)

            # Flatten the label
            ys = ys.view(-1)

            loss = criterion(outputs, ys)

            loss.backward()
            optimizer.step()

            epoch_loss += outputs.shape[0] * loss.item()
            iteration_loss += outputs.shape[0] * loss.item()
            
            # Also calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            
            correct_pred += (predicted == ys).sum().item()
            train_total += ys.size(0)


            if i % 1e2 == 0:

                round_time = time.time()
                duration = round(((round_time - start_time) / 60), 0) # To convert to minutes
                start_time = time.time()

                perc = round((i / total_train_len * 100), 2)

                iteration_loss = round((iteration_loss / 1e4), 2)
                
                accuracy = np.round((correct_pred / train_total * 100), 2)

                display(HTML(
                """<tr>
                <td>{}</td>
                <td>{}</td>
                <td>{}</td>
                <td>{}</td>
                </tr>""".format(str(epoch + 1), str(perc), str(iteration_loss), str(duration))
                ))

                iteration_loss = 0.0

    loss_history.append(epoch_loss)

    print(f'Epoch {str(epoch + 1)} Train loss: {str(epoch_loss)}.')

display(HTML('</tbody></table>'))        
print('Finished training')

Epoch,Percentage,Loss,Accuracy,Time


KeyboardInterrupt: 